In [ ]:
# Kemal Uǧur Öncel 2020204027

#Q1

#index=[keyword,[url1,url2,...,],# of links],[[keyword2,[url3,url4,...,],# of links]]
def union(list1, list2):
  for element in list2:
    if element not in list1:
      list1.append(element)
  return list1

def addToIndex(index,keyword,url): #modified add to index according to new index
  keyword_exists=False
  for cell in index:
    if cell[0]==keyword:
      union(cell[1],[url])
      cell[2]=len(cell[1])
      keyword_exists=True
      break
  if not keyword_exists:
    index.append([keyword,[url],1])
  return index

def getPage(url):
  try:
    import urllib.request
    page = urllib.request.urlopen(url).read()
    page = page.decode("utf-8")
    return page
  except:
    return ""

def addPageToIndex(index, url,content):
  contentList=content.split()
  for element in contentList:
    addToIndex(index,element,url)
  return index

def getNextTarget(page):
  startLink=page.find('<a href=')
  if startLink==-1:
    return None,0
  startQuote = page.find('"', startLink+1)
  endQuote= page.find('"', startQuote+1 )
  url= page[startQuote+1: endQuote]
  return url, endQuote

def getAllLinks(page):
  urlList=[]
  while True:
    myUrl, lastQuote = getNextTarget(page)
    if myUrl:
      page=page[lastQuote:]
      urlList.append(myUrl)
    else:
      break
  return urlList

def crawlWeb(seed):
  tocrawl=[seed]
  crawled=[]
  index=[] #index structure is modified
  graph={}
  while tocrawl:
    page=tocrawl.pop()
    if page not in crawled:
      content=getPage(page)
      addPageToIndex(index,page,content)
      graph[page]=getAllLinks(page)
      union(tocrawl,getAllLinks(getPage(page)))
      crawled.append(page)
  return index,graph

def computeRanks (graph) :
  d = 0.8
  N = len( graph )
  numloops = 10
  ranks = {}
  for page in graph :
    ranks [ page ] = 1/ N

  for i in range (0 , numloops ) :
    newranks = {}
    for page in graph :
      newrank = (1-d ) / N
      for node in graph :
        if page in graph [ node ]:
          newrank = newrank + d*(ranks[node]/len(graph(node)))
      newranks [ page ] = newrank
    ranks = newranks
  return newranks

index,graph=crawlWeb("https://searchengineplaces.com.tr/") # assigning index, graph and ranks
ranks=computeRanks(graph)

def lookUpPageRank(index,keyword,ranks):
  readytoSort={}
  keywords = [cell[0] for cell in index] # it puts all the keywords of the index to a list.
  if keyword in keywords: # checking if the keyword is already exist in index
    for cell in index:
      if keyword==cell[0]: # it will find which cell is corresponds to the given keyword
        unsortedList=cell[1] # getting all the links of the corresponding cell
        numberOfLinks=cell[2]
        for links in unsortedList:
          if links in ranks:
            readytoSort[links]=ranks[links]

    sortedList = sorted(readytoSort, key=readytoSort.get, reverse=True)
    print(f"Keyword: {keyword}")
    print(f"{numberOfLinks} result(s) found.")
    print("Urls:")
    for url in sortedList:
      print(url)
  else:
    return "No results found"

def lookUpLink(index,link):
  keywords=[] # creating empty list to store keywords
  keyword_count=0 # initializing counter
  link_found=False
  for cell in index:
    if link in cell[1]:
      keywords=union(keywords,[cell[0]])# if the given link exists in the index it add the keyword of the cell to keywords list.
      keyword_count+=1
      link_found=True
  if link_found:
    print(f"Link: {link}")
    print(f"{keyword_count} keyword(s) found.")
    for kw in keywords:
     print(kw)
  else:
    return "No results found"

#lookUpPageRank(index,"<!DOCTYPE",ranks)

lookUpLink(index,"http://www.searchengineplaces.com.tr/oktayrecommends.html")



